In [1]:
import copy
import random

def generate_city_query(tokens, ner_labels, city):
    copy_tokens = copy.deepcopy(tokens)
    copy_ner_labels = copy.deepcopy(ner_labels)

    city_index = copy_tokens.index("{city}")
    copy_tokens.pop(city_index)
    copy_ner_labels.pop(city_index)
    if random.random() > 0.5:
        city_tokens = city.split(" ")
        copy_tokens[city_index:city_index] = city_tokens
        ner_tokens = ["B-city"] + ["I-city"] * (len(city_tokens) - 1)
        copy_ner_labels[city_index:city_index] = ner_tokens
    else:
        city_tokens = ["thành", "phố"] + city.split(" ") if city in ["Hà Nội", "Hồ Chí Minh", "Cần Thơ", "Hải Phòng", "Đà Nẵng"] else ["tỉnh"] + city.split(" ")
        ner_tokens = ["O", "O", "B-city"] + ["I-city"] * (len(city_tokens) - 3) if city in ["Hà Nội", "Hồ Chí Minh", "Cần Thơ", "Hải Phòng", "Đà Nẵng"] else ["O", "B-city"] + ["I-city"] * (len(city_tokens) - 2)
        copy_tokens[city_index:city_index] = city_tokens
        copy_ner_labels[city_index:city_index] = ner_tokens

    return copy_tokens, copy_ner_labels

def generate_district_query(tokens, ner_labels, district):
    copy_tokens = copy.deepcopy(tokens)
    copy_ner_labels = copy.deepcopy(ner_labels)

    district_index = copy_tokens.index("{district}")
    copy_tokens.pop(district_index)
    copy_ner_labels.pop(district_index)

    if random.random() > 0.5:
        district_tokens = district.split(" ")
        copy_tokens[district_index:district_index] = district_tokens
        ner_tokens = ["B-district"] + ["I-district"] * (len(district_tokens) - 1)
        copy_ner_labels[district_index:district_index] = ner_tokens
    else:
        district_tokens = ["quận"] + district.split(" ") if random.random() > 0.5 else ["huyện"] + district.split(" ")
        ner_tokens = ["O"] + ["B-district"] + ["I-district"] * (len(district_tokens) - 2)
        copy_tokens[district_index:district_index] = district_tokens
        copy_ner_labels[district_index:district_index] = ner_tokens

    return copy_tokens, copy_ner_labels

def generate_query(tokens, ner_labels, **kwargs):
    copy_tokens = copy.deepcopy(tokens)
    copy_ner_labels = copy.deepcopy(ner_labels)

    key, value = next(iter(kwargs.items()))
    index = copy_tokens.index(f"{{{key}}}")
    copy_tokens.pop(index)
    copy_ner_labels.pop(index)

    key_tokens = value.split(" ")
    copy_tokens[index:index] = key_tokens
    ner_tokens = ["B-" + key] + ["I-" + key] * (len(key_tokens) - 1)
    copy_ner_labels[index:index] = ner_tokens

    return copy_tokens, copy_ner_labels
    

In [2]:
import pandas as pd
import pymongo
import random
import re 
import json

# ask city
templates = [
    "Tôi muốn tìm nhà ở {city}",
    "Tìm nhà ở {city}",
    "Có bất động sản ở {city} không?",
    "Tìm bất động sản ở {city}",
    "Tôi cần mua nhà ở {city}",
    "Cung cấp cho tôi thông tin về nhà ở {city}",
    "Có nhà nào ở {city} không?",
    "Tôi muốn mua nhà ở {city}",
    "Cho thông tin về nhà ở {city} với",
    "Cho tôi biết nhà đang bán gần {city} đi",
    "Trong {city} có nhà nào bán không?",
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city, "district": collection.distinct("district", {"city": city})}
    for city in cities
]
df = json.load(open("data/Sample.json"))

rows = []
for city in cities:
    for template in templates:
        tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
        ner_labels = ["O"] * len(tokens)
        tokens, ner_labels = generate_city_query(tokens, ner_labels, city)
        row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

        rows.append(row)
rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=100)

rows = rows.to_dict("records")
df = rows + df

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(df, f, ensure_ascii=False, indent=4)
print(len(df))

135


In [3]:
import pandas as pd
import pymongo
import random
import json

# ask district and city
templates = [
    "Tôi muốn tìm nhà ở {district}, {city}",
    "Tìm nhà ở {district}, {city}",
    "Có bất động sản ở {district}, {city} không?",
    "Tìm bất động sản ở {district}, {city}",
    "Tôi cần mua nhà ở {district}, {city}",
    "Cung cấp cho tôi thông tin về nhà ở {district}, {city}",
    "Có nhà nào ở {district}, {city} không?",
    "Tôi muốn mua nhà ở {district}, {city}",
    "Cho thông tin về nhà ở {district}, {city} với",
    "Cho tôi biết nhà đang bán gần {district}, {city} đi",
    "Trong {district}, {city} có nhà nào bán không?",
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city, "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []

for city in cities:
    for district in collection.distinct("district", {"city": city}):
        for template in templates:
            tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
            ner_labels = ["O"] * len(tokens)
            
            tokens, ner_labels = generate_city_query(tokens, ner_labels, city)
            tokens, ner_labels = generate_district_query(tokens, ner_labels, district)

            row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

            rows.append(row)

rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=200)

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

335


In [4]:
import pandas as pd
import pymongo
import random

templates_city = [
    "Tôi muốn tìm {type_of_land} ở {city}",
    "Tìm {type_of_land} ở {city}",
    "Có {type_of_land} ở {city} không?",
    "Tôi cần mua {type_of_land} ở {city}",
    "Cung cấp cho tôi thông tin về {type_of_land} ở {city}",
    "Có {type_of_land} nào ở {city} không?",
    "Tôi muốn mua {type_of_land} ở {city}",
    "Cho thông tin về {type_of_land} ở {city} với",
    "Cho tôi biết {type_of_land} đang bán gần {city} đi",
    "Trong {city} có {type_of_land} nào bán không?",
]

templates_district = [
    "Tôi muốn tìm {type_of_land} ở {district}, {city}",
    "Tìm {type_of_land} ở {district}, {city}",
    "Có {type_of_land} ở {district}, {city} không?",
    "Tôi cần mua {type_of_land} ở {district}, {city}",
    "Cung cấp cho tôi thông tin về {type_of_land} ở {district}, {city}",
    "Có {type_of_land} nào ở {district}, {city} không?",
    "Tôi muốn mua {type_of_land} ở {district}, {city}",
    "Cho thông tin về {type_of_land} ở {district}, {city} với",
    "Cho tôi biết {type_of_land} đang bán gần {district}, {city} đi",
    "Trong {district}, {city} có {type_of_land} nào bán không?",
]

types_land = ["đất", "chung cư", "chung cư mini", "căn hộ chung cư", "kho", "nhà xưởng", "biệt thự", "nhà mặt phố", "nhà riêng", "nhà phố thương mại", "shophouse", "trang trại", "khu nghỉ dưỡng", "đất nền dự án"]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city, "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []

for city in cities:
    for district in collection.distinct("district", {"city": city}):
        for type_land in types_land:
            for template in templates_district:
                tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
                ner_labels = ["O"] * len(tokens)

                tokens, ner_labels = generate_city_query(tokens, ner_labels, city)
                tokens, ner_labels = generate_district_query(tokens, ner_labels, district)
                tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=type_land)
                

                row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}                

for city in cities:
    for type_land in types_land:
        for template in templates_city:
            tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
            ner_labels = ["O"] * len(tokens)
            
            tokens, ner_labels = generate_city_query(tokens, ner_labels, city)
            tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=type_land)
            
            row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

            rows.append(row)
            

rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=300)

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

635


In [5]:
import pandas as pd
import pymongo
import random
import copy

legals = ["sổ đỏ", "sổ hồng", "giấy phép", "pháp lý rõ ràng", "pháp lý đầy đủ", "đầy đủ pháp lý", "giấy tờ xây dựng", "hợp đồng góp vốn", "hợp đồng mua bán", "hợp đồng đặt cọc", "SĐCC", "sổ đổ chính chủ", "vi bằng"]

templates = [
    "Tôi muốn tìm nhà có {legal}",
    "Tìm nhà có {legal}",
    "Có nhà nào có {legal} không?",
    "Tôi cần mua nhà có {legal}",
    "Cung cấp cho tôi thông tin về nhà có {legal}",
    "Có nhà nào có {legal} không?",
    "Tôi muốn mua nhà có {legal}",
    "Cho thông tin về nhà có {legal} với",
    "Cho tôi biết nhà có {legal} đang bán đi",
    "Có nhà nào có {legal} bán không?",
    "Nhà nào ở {city} có {legal}?",
    "Nhà nào ở {district}, {city} có {legal}?",
    "Nhà nào có {type_of_land} ở {city} có {legal}?",
    "Nhà nào có {type_of_land} ở {district}, {city} có {legal}?",
    "Tôi muốn tìm nhà có {legal} ở {city}",
    "Tìm nhà có {legal} ở {city}",
    "Cho thông tin về {type_of_land} có {legal} ở {city}",
    "Trong {city} có {type_of_land} nào có {legal} không?",
    "Ở {district}, {city} có {type_of_land} nào có {legal} không?",
    "Bạn có thông tin về bất động sản có {legal} ở {city} không?",
    "Có {type_of_land} ở {city} có {legal} ?",
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city
        , "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []

for legal in legals:
    for template in templates:
        tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
        ner_labels = ["O"] * len(tokens)
        
        check = {"legal": legal, "city": None, "district": None, "type_of_land": None}
        
        if "city" in template:
            check["city"] = random.choice(cities)
            tokens, ner_labels = generate_city_query(tokens, ner_labels, check["city"])

        if "district" in template:
            check["district"] = random.choice(district_dict[cities.index(check["city"])]["district"])
            tokens, ner_labels = generate_district_query(tokens, ner_labels, check["district"])

        if "type_of_land" in template:
            check["type_of_land"] = random.choice(types_land)
            tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=check["type_of_land"])
        if "legal" in template:
            tokens, ner_labels = generate_query(tokens, ner_labels, legal=legal)
        
        row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

        rows.append(row)



rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=400) if len(rows) > 400 else rows

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

895


In [6]:
import pandas as pd
import pymongo
import random
import copy

templates = [
    "Tìm nhà ở {city} giá dưới {max_price}",
    "Có {type_of_land} ở {city} giá từ {min_price} đến {max_price} không?",
    "Có nhà ở {city} giá dưới {max_price} không?",
    "Cung cấp cho tôi thông tin về nhà ở {district}, {city} giá từ {min_price} đến {max_price}",
    "Có nhà ở {district}, {city} giá từ {min_price} đến {max_price} không?",
    "Bất động sản ở {city} giá từ {min_price} đến {max_price} có không?",
    "Tôi muốn mua nhà ở {city} giá từ {min_price} đến {max_price}",
    "Cho thông tin về nhà ở {city} giá không dưới {min_price}",
    "Cho tôi biết {type_of_land} đang bán gần {city} giá từ {min_price} đến {max_price} đi",
    "Trong {city} có nhà nào bán giá dưới {max_price} không?",
    "Nhà nào ở {district}, {city} giá từ {min_price} đến {max_price} với {legal}?",
    "{type_of_land} nào ở {city} giá trên {min_price} mà có {legal}?",
    "Tôi muốn tìm nhà ở {city} giá từ {min_price} đến {max_price}",
    "Bạn hãy cho tôi thông tin về nhà ở {city} dưới {max_price} với {legal}",
    "Tôi cần mua nhà ở {city} giá trên {min_price}",
    "Tìm {type_of_land} ở {city} giá trên {min_price}",
    "Có nhà ở {city} giá không dưới {min_price} không?",
    "Cho thông tin về {type_of_land} ở {city} giá không dưới {min_price}",
    "Trong {city} có {type_of_land} nào bán giá trên {min_price} không?",
    "Nhà nào ở {district}, {city} giá không quá {max_price}?",
    "{type_of_land} nào ở {city} giá trên {min_price} mà có {legal}?",
    "{type_of_land} nào ở {city} giá thấp hơn {max_price} mà có {legal}?",
    "Tìm nhà với {legal} giá dưới {max_price} ở {city}",
    "Tìm nhà với {legal} giá thấp hơn {max_price} ở {city}",
    "Có {type_of_land} ở {city} giá dưới {max_price} không?",
    "Có nhà ở {city} giá trên {min_price} không?",
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city
        , "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []

for template in templates:
    for city in cities:
        tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
        ner_labels = ["O"] * len(tokens)
        random1 = random.random()
        check = {
            "city": None,
            "district": None,
            "type_of_land": None,
            "legal": None,
            "min_price": None,
            "max_price": None,
        }

        if "city" in template:
            check["city"] = random.choice(cities)
            tokens, ner_labels = generate_city_query(tokens, ner_labels, check["city"])
        if "district" in template:
            check["district"] = random.choice(district_dict[cities.index(check["city"])]["district"])
            tokens, ner_labels = generate_district_query(tokens, ner_labels, check["district"])
        if "type_of_land" in template:
            check["type_of_land"] = random.choice(types_land)
            tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=check["type_of_land"])
        if "legal" in template:
            check["legal"] = random.choice(legals)
            tokens, ner_labels = generate_query(tokens, ner_labels, legal=check["legal"])
        if "min_price" in template:
            check["min_price"] = str(random.randint(1, 6)) + " tỷ" if random1 > 0.5 else str(random.randint(100, 500)) + " triệu"
            tokens, ner_labels = generate_query(tokens, ner_labels, min_price=check["min_price"])

        if "max_price" in template:
            check["max_price"] = str(random.randint(6, 11)) + " tỷ" if random1 > 0.5 else str(random.randint(500, 1000)) + " triệu"
            tokens, ner_labels = generate_query(tokens, ner_labels, max_price=check["max_price"])

        row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

        rows.append(row)

rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=600)

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

1495


In [7]:
import pandas as pd
import pymongo
import random
import copy

templates = [
    "Tìm nhà ở {city} có diện tích trên {min_acreage}",
    "Có {type_of_land} ở {city} có diện tích từ {min_acreage} đến {max_acreage} không?",
    "Có nhà ở {city} có diện tích từ {min_acreage} đến {max_acreage} không?",
    "Cung cấp cho tôi thông tin về nhà ở {district}, {city} có diện tích trên {min_acreage}",
    "Có nhà ở {district}, {city} có diện tích từ {min_acreage} đến {max_acreage} không?",
    "Bất động sản ở {city} có diện tích từ {min_acreage} đến {max_acreage} có không?",
    "Tôi muốn mua nhà ở {district}, {city} có diện tích từ {min_acreage} đến {max_acreage}",
    "Cho thông tin về nhà ở {city} có diện tích từ không dưới {min_acreage}",
    "Bạn có nhà nào có diện tích từ {min_acreage} đến {max_acreage} không?",
    "Nhà nào ở {district}, {city} có diện tích từ {min_acreage} đến {max_acreage} với {legal}?",
    "{type_of_land} nào ở {city} có diện tích từ {min_acreage} đến {max_acreage} mà có {legal}?",
    "Tôi muốn tìm nhà ở {city} có diện tích dưới {max_acreage}",
    "Bạn hãy cho tôi thông tin về nhà ở {city} có diện tích trên {min_acreage} với {legal}",
    "Tìm nhà có giá dưới {max_price} và diện tích từ {min_acreage} đến {max_acreage}",
    "Có nhà nào ở {city} có giá dưới {max_price} và diện tích trên {min_acreage} không?",
    "Cho tôi thông tin {type_of_land} với {legal} có giá từ {min_price} đến {max_price} và diện tích từ {min_acreage} đến {max_acreage}",
    "Trong {city} có {type_of_land} nào có giá không quá {max_price} và diện tích dưới {max_acreage} không?",
    "Tìm nhà với {legal} giá dưới {max_price} và diện tích không quá {max_acreage} ở {city}",
    "Tìm nhà với {legal} giá thấp hơn {max_price} và diện tích nhỏ hơn {max_acreage} ở {city}",
    "Có {type_of_land} ở {city} giá dưới {max_price} và diện tích trên {min_acreage} không?",
    "Có nhà ở {city} giá trên {min_price} và diện tích dưới {max_acreage} không?",
    "Cho tôi thông tin về {type_of_land} ở {city} giá không dưới {min_price} và diện tích duới {max_acreage}",
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city
        , "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []

for template in templates:
    for city in cities:
        tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
        ner_labels = ["O"] * len(tokens)
        random1 = random.random()
        check = {
            "city": None,
            "district": None,
            "type_of_land": None,
            "min_price": None,
            "max_price": None,
            "min_acreage": None,
            "max_acreage": None,
        }

        if "city" in template:
            check["city"] = city
            tokens, ner_labels = generate_city_query(tokens, ner_labels, check["city"])
        if "district" in template:
            check["district"] = random.choice(district_dict[cities.index(check["city"])]["district"])
            tokens, ner_labels = generate_district_query(tokens, ner_labels, check["district"])
        if "type_of_land" in template:
            check["type_of_land"] = random.choice(types_land)
            tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=check["type_of_land"])
        if "legal" in template:
            check["legal"] = random.choice(legals)
            tokens, ner_labels = generate_query(tokens, ner_labels, legal=check["legal"])
        if "min_price" in template:
            check["min_price"] = str(random.randint(1, 6)) + " tỷ" if random1 > 0.5 else str(random.randint(100, 500)) + " triệu"
            tokens, ner_labels = generate_query(tokens, ner_labels, min_price=check["min_price"])
        if "max_price" in template:
            check["max_price"] = str(random.randint(6, 11)) + " tỷ" if random1 > 0.5 else str(random.randint(500, 1000)) + " triệu"
            tokens, ner_labels = generate_query(tokens, ner_labels, max_price=check["max_price"])
        if "min_acreage" in template:
            check["min_acreage"] = str(random.randint(10, 60)) + " m2"
            tokens, ner_labels = generate_query(tokens, ner_labels, min_acreage=check["min_acreage"])
        if "max_acreage" in template:
            check["max_acreage"] = str(random.randint(60, 100)) + " m2"
            tokens, ner_labels = generate_query(tokens, ner_labels, max_acreage=check["max_acreage"])

        row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}

        rows.append(row)


rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=600)

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

2095


In [8]:
import pandas as pd
import pymongo
import random
import copy

templates = [
    "Tìm nhà ở {district}, {city} hướng {house_direction}",
    "Có {type_of_land} ở {district}, {city} có ban công hướng {balcony_direction} không?",
    "Có nhà ở {city} hướng {house_direction} không?",
    "Cung cấp cho tôi thông tin về nhà có diện tích trên {min_acreage} và ban công hướng {balcony_direction}",
    "Có nhà ở {district}, {city} hướng {house_direction} không?",
    "Bất động sản ở {city} hướng {house_direction} có không?",
    "Tôi muốn mua nhà ở {district}, {city} hướng {house_direction} với {legal}",
    "Cho thông tin về nhà ở {city} hướng {house_direction} với",
    "Bạn có nhà nào hướng {house_direction} không?",
    "Nhà nào ở {district}, {city} ban công hướng {balcony_direction} với {legal}?",
    "{type_of_land} nào ở {city} hướng {house_direction} mà có {legal}?",
    "Tôi muốn tìm nhà ở {city} hướng {house_direction} giá từ {min_price} đến {max_price}",
    "Bạn hãy cho tôi thông tin về nhà ở {city} hướng {house_direction} với {legal}",
    "Tìm nhà có giá dưới {max_price} và hướng {house_direction}",
    "Có nhà nào ở {city} có giá dưới {max_price} và hướng {house_direction} không?",
    "Cho tôi thông tin {type_of_land} với {legal} có giá từ {min_price} đến {max_price} và hướng {house_direction}",
    "Trong {city} có {type_of_land} nào có giá không quá {max_price} và hướng {house_direction} không?",
    "Bạn có thông tin về bất động sản hướng {house_direction} ở {city} không?"
]

collection = pymongo.MongoClient("mongodb://localhost:27017")["real_estate"]["batdongsan"]

cities = collection.distinct("city")
district_dict = [
    {"city": city
        , "district": collection.distinct("district", {"city": city})}
    for city in cities
]

data = json.load(open("data/train_data.json"))
rows = []
types_land = ["đất", "chung cư", "chung cư mini", "căn hộ chung cư", "kho", "nhà xưởng", "biệt thự", "nhà mặt phố", "nhà riêng", "nhà phố thương mại", "shophouse", "trang trại", "khu nghỉ dưỡng", "đất nền dự án"]
legals = ["sổ đỏ", "sổ hồng", "giấy phép", "pháp lý rõ ràng", "pháp lý đầy đủ", "đầy đủ pháp lý", "giấy tờ xây dựng", "hợp đồng góp vốn", "hợp đồng mua bán", "hợp đồng đặt cọc", "SĐCC", "sổ đổ chính chủ", "vi bằng"]

house_directions = collection.distinct("house_direction")
balcony_directions = collection.distinct("balcony_direction")

for template in templates:
    for house_direction in house_directions:
        for balcony_direction in balcony_directions:
            for city in cities:
                tokens = re.findall(r"\{[^}]*\}|\w+|[^\w\s]", template)
                ner_labels = ["O"] * len(tokens)
                random1 = random.random()
                check = {
                    "city": None,
                    "district": None,
                    "type_of_land": None,
                    "legal": None,
                    "min_price": None,
                    "max_price": None,
                    "min_acreage": None,
                    "max_acreage": None,
                    "house_direction": None,
                    "balcony_direction": None,
                }

                if "city" in template:
                    check["city"] = city
                    tokens, ner_labels = generate_city_query(tokens, ner_labels, check["city"])
                if "district" in template:
                    check["district"] = random.choice(district_dict[cities.index(check["city"])]["district"])
                    tokens, ner_labels = generate_district_query(tokens, ner_labels, check["district"])
                if "type_of_land" in template:
                    check["type_of_land"] = random.choice(types_land)
                    tokens, ner_labels = generate_query(tokens, ner_labels, type_of_land=check["type_of_land"])
                if "legal" in template:
                    check["legal"] = random.choice(legals)
                    tokens, ner_labels = generate_query(tokens, ner_labels, legal=check["legal"])
                if "min_price" in template:
                    check["min_price"] = str(random.randint(1, 6)) + " tỷ" if random1 > 0.5 else str(random.randint(100, 500)) + " triệu"
                    tokens, ner_labels = generate_query(tokens, ner_labels, min_price=check["min_price"])
                if "max_price" in template:
                    check["max_price"] = str(random.randint(6, 11)) + " tỷ" if random1 > 0.5 else str(random.randint(500, 1000)) + " triệu"
                    tokens, ner_labels = generate_query(tokens, ner_labels, max_price=check["max_price"])
                if "min_acreage" in template:
                    check["min_acreage"] = str(random.randint(10, 60)) + " m2"
                    tokens, ner_labels = generate_query(tokens, ner_labels, min_acreage=check["min_acreage"])
                if "max_acreage" in template:
                    check["max_acreage"] = str(random.randint(60, 100)) + " m2"
                    tokens, ner_labels = generate_query(tokens, ner_labels, max_acreage=check["max_acreage"])
                if "house_direction" in template:
                    check["house_direction"] = house_direction
                    tokens, ner_labels = generate_query(tokens, ner_labels, house_direction=check["house_direction"])
                if "balcony_direction" in template:
                    check["balcony_direction"] = balcony_direction
                    tokens, ner_labels = generate_query(tokens, ner_labels, balcony_direction=check["balcony_direction"])

                row = {"Tokens": tokens, "Intent": "directed_query", "ner_labels": ner_labels}
                rows.append(row)

rows = pd.DataFrame(rows)
rows.drop_duplicates(subset="Tokens", inplace=True)
rows = rows.sample(n=400)

rows = rows.to_dict("records")
data = rows + data

with open("data/train_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(len(data))

2495
